In [2]:
#load in podcast df
import pickle
import time
import pandas as pd

floc = '/home/bmassi/Dropbox/professional/Insight/data/'
fname = 'podcast_df_subset_'+'1528942958'+'.pkl'
with open(floc+fname,'rb') as fid:
    podcast_df_subset = pickle.load(fid)
podcast_df_subset.drop_duplicates(['feedUrl'])

,artistId,artistName,artistViewUrl,artworkUrl100,artworkUrl30,artworkUrl60,artworkUrl600,collectionCensoredName,collectionExplicitness,collectionHdPrice,...,trackExplicitness,trackHdPrice,trackHdRentalPrice,trackId,trackName,trackPrice,trackRentalPrice,trackViewUrl,wrapperType,w2v
2,NaN,OK Beast,NaN,https://is2-ssl.mzstatic.com/image/thumb/Music...,https://is2-ssl.mzstatic.com/image/thumb/Music...,https://is2-ssl.mzstatic.com/image/thumb/Music...,https://is2-ssl.mzstatic.com/image/thumb/Music...,A+ Anime,explicit,0,...,explicit,0,0,1238028226,A+ Anime,0.0,0,https://itunes.apple.com/us/podcast/a-anime/id...,track,"[11.925855787883748, 11.963884158233876, 11.94..."
20,NaN,OMGLOLJK,NaN,https://is1-ssl.mzstatic.com/image/thumb/Music...,https://is1-ssl.mzstatic.com/image/thumb/Music...,https://is1-ssl.mzstatic.com/image/thumb/Music...,https://is1-ssl.mzstatic.com/image/thumb/Music...,A.D.D. Podcast with Jason Barr,explicit,0,...,explicit,0,0,393281425,A.D.D. Podcast with Jason Barr,0.0,0,https://itunes.apple.com/us/podcast/a-d-d-podc...,track,"[10.237858041074675, 10.261172640791369, 10.21..."
47,NaN,Asian American Writers' Workshop,NaN,https://is4-ssl.mzstatic.com/image/thumb/Music...,https://is4-ssl.mzstatic.com/image/thumb/Music...,https://is4-ssl.mzstatic.com/image/thumb/Music...,https://is4-ssl.mzstatic.com/image/thumb/Music...,AAWW Radio: New Asian American Writers & Liter...,cleaned,0,...,cleaned,0,0,1297736720,AAWW Radio: New Asian American Writers & Liter...,0.0,0,https://itunes.apple.com/us/podcast/aaww-radio...,track,"[4.949070514835612, 4.9728138054938835, 4.9470..."
55,NaN,AFDE Show,NaN,https://is3-ssl.mzstatic.com/image/thumb/Music...,https://is3-ssl.mzstatic.com/image/thumb/Music...,https://is3-ssl.mzstatic.com/image/thumb/Music...,https://is3-ssl.mzstatic.com/image/thumb/Music...,Abbatacola & Ferrari Discuss Everything,explicit,0,...,explicit,0,0,1266063931,Abbatacola & Ferrari Discuss Everything,0.0,0,https://itunes.apple.com/us/podcast/abbatacola...,track,"[10.721073351101916, 10.778949162951442, 10.71..."
69,NaN,Abe's Ukulele Podcast,NaN,https://is4-ssl.mzstatic.com/image/thumb/Music...,https://is4-ssl.mzstatic.com/image/thumb/Music...,https://is4-ssl.mzstatic.com/image/thumb/Music...,https://is4-ssl.mzstatic.com/image/thumb/Music...,Abe's Ukulele Podcast,cleaned,0,...,cleaned,0,0,1268431796,Abe's Ukulele Podcast,0.0,0,https://itunes.apple.com/us/podcast/abes-ukule...,track,"[2.6152033439180635, 2.5846214996415475, 2.564..."
81,465018360.0,Abnormal Entertainment,https://itunes.apple.com/us/artist/abnormal-en...,https://is2-ssl.mzstatic.com/image/thumb/Music...,https://is2-ssl.mzstatic.com/image/thumb/Music...,https://is2-ssl.mzstatic.com/image/thumb/Music...,https://is2-ssl.mzstatic.com/image/thumb/Music...,"Abnormal Entertainment - Podcasts, Movies, Com...",explicit,0,...,explicit,0,0,460790820,"Abnormal Entertainment - Podcasts, Movies, Com...",0.0,0,https://itunes.apple.com/us/podcast/abnormal-e...,track,"[5.38304043808641, 5.381466970426353, 5.321859..."
98,NaN,Taylor Hughes,NaN,https://is5-ssl.mzstatic.com/image/thumb/Music...,https://is5-ssl.mzstatic.com/image/thumb/Music...,https://is5-ssl.mzstatic.com/image/thumb/Music...,https://is5-ssl.mzstatic.com/image/thumb/Music...,About to Break,cleaned,0,...,cleaned,0,0,1190260966,About to Break,0.0,0,https://itunes.apple.com/us/podcast/about-to-b...,track,"[5.623342402286933, 5.616956935154565, 5.56662..."
103,NaN,AboutBeverages.com,NaN,https://is3-ssl.mzstatic.com/image/thumb/Music...,https://is3-ssl.mzstatic.com/image/thumb/Music...,https://is3-ssl.mzstatic.com/image/thumb/Music...,https://is3-ssl.mzstatic.com/image/thumb/Music...,AboutBeverages.com - Podcast,cleaned,0,...,cleaned,0,0,306133017,AboutBeverages.com - Podcast,0.0,0,https://itunes.apple.com/us/podcast/aboutbever...,track,"[6.613417222764755, 6.646760389539932, 6.63293..."
110,NaN,Chuck Clough and Ronnie Hirschberg,NaN,https://is3-ssl.mzstatic.com/image/thumb/Music...,https://is3-ssl.mzstatic.com/im

In [ ]:
#load in word2vec, and define functions for parsing tokenized episode summaries.
import gensim
import numpy as np

floc = '/home/bmassi/Dropbox/professional/Insight/data/'
fname = 'GoogleNews-vectors-negative300.bin'

word2vec = gensim.models.KeyedVectors.load_word2vec_format(floc+fname, binary=True)

print(word2vec['apples'])

In [ ]:
#define functions for vectorizing words

#takes list of words, safely computes average word2vec. 
def ep2vec(episode):
    n = 0
    a = np.array([300,])
    for w in episode:
        try:
            v = word2vec[w]
            if((np.isnan(v).any() + np.isinf(v).any()) == 0):
                a = np.add(a,v)
                n += 1
        except KeyError as e:
            pass
    if(n==0):
        return np.full([300,], np.nan)
    avg = a/n
    return avg

#takes lists of episodes, vectorizes them, then averages the vectors
def pod2vec(episodes):
    return np.nanmean(np.asarray([ep2vec(e) for e in episodes]),axis=0)

In [ ]:
#Fit a gaussian mixture model for clustering
import numpy as np
from sklearn import mixture

np.array([v for v in podcast_df_subset['w2v'].get_values()]).shape#training examples
gmm = mixture.GaussianMixture(n_components=15)
podcast_topic_vec = np.array()
gmm.fit(podcast_topic_vec)